In [2]:
!unzip Data.zip -d Data

Archive:  Data.zip
   creating: Data/Data/
  inflating: Data/Data/GBvideos.csv  
  inflating: Data/Data/CA_category_id.json  
  inflating: Data/Data/US_category_id.json  
  inflating: Data/Data/USvideos.csv  
  inflating: Data/Data/CAvideos.csv  
  inflating: Data/Data/GB_category_id.json  


#Extracting Data  

In [6]:
import pandas as pd
import numpy as np
import json

In [7]:

#load all the datasets 
df1 = pd.read_csv('/content/Data/Data/USvideos.csv')
df2 = pd.read_csv('/content/Data/Data/CAvideos.csv')
df3 = pd.read_csv('/content/Data/Data/GBvideos.csv')

#load the datasets containing the category names
data1 = json.load(open('/content/Data/Data/US_category_id.json'))
data2 = json.load(open('/content/Data/Data/CA_category_id.json'))
data3 = json.load(open('/content/Data/Data/GB_category_id.json'))

In [8]:
def category_extractor(data):
    i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    i_d = list(map(int, i_d))
    category = zip(i_d, title)
    category = dict(category)
    return category


In [9]:
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))

In [10]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df = df.drop_duplicates('video_id')

In [11]:
entertainment = df[(df['category_title'] == 'Entertainment') | (df['category_title']=='Science & Technology')]['title']
entertainment = entertainment.tolist()

# Data Preprocessing

In [12]:
entertainment[:5]

['The Trump Presidency: Last Week Tonight with John Oliver (HBO)',
 'Nickelback Lyrics: Real or Fake?',
 'I Dare You: GOING BALD!?',
 '2 Weeks with iPhone X',
 'Roy Moore & Jeff Sessions Cold Open - SNL']

In [14]:
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

In [15]:
corpus[:10]

['the trump presidency last week tonight with john oliver hbo',
 'nickelback lyrics real or fake',
 'i dare you going bald',
 '2 weeks with iphone x',
 'roy moore  jeff sessions cold open  snl',
 '5 ice cream gadgets put to the test',
 'spoilers shiva saves the day talked about scene ep 804  the walking dead',
 'the new spotmini',
 'homemade electric airplane',
 'the making of helas headdress from thor ragnarok']

In [45]:
from keras.preprocessing.text import Tokenizer
token = Tokenizer()
def train_label(data):
  
  words = token.fit_on_texts(data)
  total_words = len(token.word_index)+1
  input_data = []
  for line in data:
    l = token.texts_to_sequences([line])[0]
    for i in range(1,len(l)):
      d = l[:i+1]
      input_data.append(d)
  return input_data,total_words



In [48]:
inp,totword = train_label(corpus)

In [26]:
inp[:15]

[[1, 96],
 [1, 96, 4362],
 [1, 96, 4362, 76],
 [1, 96, 4362, 76, 359],
 [1, 96, 4362, 76, 359, 1438],
 [1, 96, 4362, 76, 359, 1438, 9],
 [1, 96, 4362, 76, 359, 1438, 9, 145],
 [1, 96, 4362, 76, 359, 1438, 9, 145, 1178],
 [1, 96, 4362, 76, 359, 1438, 9, 145, 1178, 1741],
 [6573, 1546],
 [6573, 1546, 128],
 [6573, 1546, 128, 94],
 [6573, 1546, 128, 94, 397],
 [26, 1971],
 [26, 1971, 25]]

In [28]:
maxlen = max([len(x) for x in inp])

In [29]:
maxlen

27

In [31]:
from keras.preprocessing.sequence import pad_sequences
paddata = pad_sequences(inp, maxlen=maxlen,  padding='pre')

In [32]:
paddata

array([[   0,    0,    0, ...,    0,    1,   96],
       [   0,    0,    0, ...,    1,   96, 4362],
       [   0,    0,    0, ...,   96, 4362,   76],
       ...,
       [   0,    0,    0, ...,   13, 2486,   23],
       [   0,    0,    0, ..., 2486,   23,   18],
       [   0,    0,    0, ...,   23,   18,  110]], dtype=int32)

In [33]:
X = paddata[:,:-1]
Y = paddata[:,-1]

In [34]:
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,   96],
       [   0,    0,    0, ...,    1,   96, 4362],
       ...,
       [   0,    0,    0, ...,   20,   13, 2486],
       [   0,    0,    0, ...,   13, 2486,   23],
       [   0,    0,    0, ..., 2486,   23,   18]], dtype=int32)

In [85]:
Y

array([  96, 4362,   76, ...,   23,   18,  110], dtype=int32)

In [84]:
# Converting the OutPut into one hot encoding for language model

In [89]:
from keras.utils import np_utils

In [90]:
Y2 = np_utils.to_categorical(Y, num_classes = totword)

In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding,LSTM,Dropout

In [91]:
model=Sequential() 
model.add(Embedding(input_dim=totword+1,output_dim=10,input_length=maxlen-1))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(totword, activation='softmax'))

In [92]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             )

In [97]:
mf=model.fit(X, Y2,verbose=5,epochs=3,batch_size=10)

InternalError: ignored

In [82]:
def generate_text(seed_text, next_words, model, max_sequence_len):
  for _ in range(next_words):
    token_list = token.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,  padding='pre')
    predicted = model.predict(token_list, verbose=0)
  
    output_word = ''
    for word,index in token.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += ' '+output_word
  return seed_text.title()

In [83]:
generate_text('hello',5,model,maxlen)

'Hello The The The The The'

In [72]:
sent = 'hello how are you'
tok = token.texts_to_sequences([sent])[0]
pad = pad_sequences([tok], maxlen=maxlen-1,  padding='pre')

In [73]:
val = model.predict(pad)

In [74]:
val,tok

(array([[1.]], dtype=float32), [2163, 47, 99, 25])

In [67]:
tok.append(list(val[0]))
tok

[2163, array([1.], dtype=float32), [1.0]]

In [71]:
tok + list(val[0])

[2163, 1.0]